In [ ]:
!pip install PyMuPDF python-docx hazm

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 892.6/892.6 kB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.3/243.3 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 10.9 MB/s eta 0:00:00
  Created wheel for flashtext: filename=flashtext-2.7-py2.py3-none-any.whl size=9300 sha256=3c504ca8e6d4caf45a5095385b04c51cdd71c88780076f44eb98c9feac9ec5eb
  Stored 

In [ ]:
!pip install --upgrade numpy transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 61.3 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.3
    Uninstalling numpy-1.24.3:
      Successfully uninstalled numpy-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.3 requires numpy<2.0,>=1.18.5, but you have numpy 2.2.5 which is incompatible.
hazm 0.10.0 requires numpy==1.24.3, but you have numpy 2.2.5 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.2.5 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 2.2.5 which is incompatible.


In [10]:
# import fitz  # PyMuPDF
# from hazm import Normalizer, sent_tokenize
import json
import re
import numpy as np
from sentence_transformers import SentenceTransformer
from sentence_transformers.models import Pooling, Transformer
from sklearn.metrics.pairwise import cosine_similarity
from transformers import pipeline

# 1. Preparing the dataset

In [ ]:
def chunk_by_headings(pdf_path,
                     size_threshold=18,
                     max_chunk_size=300,
                     skip_first=3,
                     skip_last=2):
  doc = fitz.open(pdf_path)
  normalizer = Normalizer()
  chunks = []
  current_chunk = ""
  current_heading = ""
  heading_hierarchy = []
  total_pages = len(doc)
  unwanted_headings = [
    'آیا می¬دانید',
    'جمع¬آوری اطلاعات',
    'خود را بیازمایید',
    'آزمایش کنید',
    'فکر کنید',
    'گفت¬و¬گو کنید',
    'فعالیت',
    'وگوکنید گفت',
    'وسایل و مواد الزم:',
    'دانید آیا می',
    'جمع‌آوری اطالعات',
    'آوری اطالعات جمع',
    'هشدار',
    '+',
    '-',
    'کنید فکر',
    'گو کنید و گفت',
    'جمع',
    'آوری اطالعات',
    'ها:'
  ]

  for page_num in range(skip_first, total_pages - skip_last):
    page = doc.load_page(page_num)
    blocks = page.get_text("dict")["blocks"]

    for block in blocks:
      if "lines" in block:
        for line in block["lines"]:
          for span in line["spans"]:
            text = normalizer.normalize(span["text"])
            font_size = span["size"]
            is_bold = "bold" in span["font"].lower()

            # Detect heading
            if font_size >= size_threshold and is_bold and text.strip() not in unwanted_headings and not text.strip().endswith('فصل'):
              # Finalize current chunk if exists
              if current_chunk:
                chunks.append({
                  "text": current_chunk.strip(),
                  "heading": current_heading,
                  "hierarchy": heading_hierarchy.copy(),
                  "page": page_num + 1
                })
                current_chunk = ""

              # Update heading tracking
              if text not in heading_hierarchy:
                if len(heading_hierarchy) > 0 and font_size <= heading_hierarchy[-1]["size"]:
                  heading_hierarchy.pop()
                heading_hierarchy.append({
                  "text": text,
                  "size": font_size,
                  "page": page_num + 1
                })
              current_heading = text
            else:
              # Add normal text to current chunk
              current_chunk += " " + text

              # Split chunk if too large
              if len(current_chunk.split()) > max_chunk_size:
                chunks.append({
                  "text": current_chunk.strip(),
                  "heading": current_heading,
                  "hierarchy": heading_hierarchy.copy(),
                  "page": page_num + 1
                })
                current_chunk = ""

    # Add remaining content at page end
    if current_chunk:
      chunks.append({
        "text": current_chunk.strip(),
        "heading": current_heading,
        "hierarchy": heading_hierarchy.copy(),
        "page": page_num + 1
      })
      current_chunk = ""

  doc.close()
  return chunks

In [ ]:
pdf_path = '/content/drive/MyDrive/C906.pdf'

In [ ]:
chunks = chunk_by_headings(
    pdf_path,
    size_threshold=14,
    max_chunk_size=250,
    skip_first=8,
    skip_last=3
)

In [ ]:
# Sample output structure
for chunk in chunks:
    print(f"\nPage {chunk['page']}")
    print(f"Heading: {chunk['heading']}")
    print(f"Hierarchy: {[h['text'] for h in chunk['hierarchy']]}")
    print(f"Content: {chunk['text'][:100]}...")


Page 9
Heading: 
Hierarchy: []
Content: ، کنید، از موادی مانند سنگ، چوب، فلز‌همه چیزهایی که شما در زندگی روزمره از آنها استفاده می اند. برخی...

Page 10
Heading: 
Hierarchy: []
Content: ۲...

Page 10
Heading: اند‌برخی مواد فلزند یا از فلز ساخته‌شده
Hierarchy: ['اند\u200cبرخی مواد فلزند یا از فلز ساخته\u200cشده']
Content: بندی عنصرها به دو دسته فلز و نافلز آشنا شدید. انسان از هزاران سال پیش فلزها را در علوم هفتم با طبقه ...

Page 11
Heading: اند‌برخی مواد فلزند یا از فلز ساخته‌شده
Hierarchy: ['اند\u200cبرخی مواد فلزند یا از فلز ساخته\u200cشده']
Content: ۳ فلز مس به علت رسانایی الکتریکی زیاد، مقاومت در برابر خوردگی و قابلیت مفتول شدن، کاربرد های مسی در ...

Page 11
Heading: پذیری یکسانی ندارند‌فلزها واکنش
Hierarchy: ['پذیری یکسانی ندارند\u200cفلزها واکنش']
Content: شود. فلز مس نیز با اکسیژن  دهد و به زنگ آهن تبدیل می کندی واکنش می دانید که آهن با اکسیژن به می شود....

Page 12
Heading: پذیری یکسانی ندارند‌فلزها واکنش
Hierarchy: ['پذیری یکسانی ندارند\u200cفلزها واکنش']
Content: ۴ ) H ۲ SO

In [ ]:
def save_chunks(chunks, filename):
  normalizer = Normalizer()

  serializable_chunks = []
  for chunk in chunks:
    serializable_chunks.append({
      'text': normalizer.normalize(chunk['text']),
      'heading': normalizer.normalize(chunk.get('heading', '')),
      'page': chunk.get('page', 0)
    })

  with open(filename, 'w', encoding='utf-8') as f:
    json.dump(serializable_chunks, f, ensure_ascii=False, indent=2)

In [ ]:
save_chunks(chunks, 'persian_textbook_chunks.json')

# 2. Embedding

In [11]:
model_name = "HooshvareLab/bert-fa-base-uncased"
embedding_model = Transformer(model_name)
pooling_model = Pooling(embedding_model.get_word_embedding_dimension())
model = SentenceTransformer(modules=[embedding_model, pooling_model])

In [ ]:
# load chucks from file
def load_chunks(filename):
  with open(filename, 'r', encoding='utf-8') as f:
    chunks = json.load(f)

  required_keys = ['text']
  valid_chunks = []

  for chunk in chunks:
    if all(key in chunk for key in required_keys):
      valid_chunks.append(chunk)

  return valid_chunks

chunks_file_path = '/content/drive/MyDrive/persian_textbook_chunks.json'
chunks = load_chunks(chunks_file_path)
print(len(chunks))
# print(chunks[0])

355


In [ ]:
# create embeddings
chunk_embeddings = model.encode([chunk['text'] for chunk in chunks])
for i, chunk in enumerate(chunks):
  chunk['embedding'] = chunk_embeddings[i].tolist()

In [ ]:
len(chunks[0]['embedding'])

768

In [ ]:
# save chunks with embeddings
file_name = '/content/drive/MyDrive/textbook_chunks_with_embeddings.json'
with open(file_name, 'w', encoding='utf-8') as f:
  json.dump({
    "chunks": chunks,
    "metadata": {
      "embedding_model": model_name
    }
  }, f, ensure_ascii=False, indent=2)

# 3. Searching

In [12]:
def load_chunks_with_embeddings(file_path):
  with open(file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)
  for chunk in data['chunks']:
    chunk['embedding'] = np.array(chunk['embedding'])
  return {
    "chunks": data['chunks'],
    "metadata": data['metadata'],
  }

In [19]:
# load chunks of data from file
file_path = '/content/drive/MyDrive/textbook_chunks_with_embeddings.json'
chunks = load_chunks_with_embeddings(file_path)['chunks']

In [ ]:
def find_relevant_chunks(question, chunks, top_k=3):
  question_embedding = model.encode(question)

  scored_chunks = []
  for chunk in chunks:
    chunk_embedding = np.array(chunk['embedding'])
    similarity = cosine_similarity([question_embedding], [chunk_embedding])[0][0]
    scored_chunks.append({
      **chunk,
      "similarity_score": float(similarity)
    })

  scored_chunks.sort(key=lambda x: x["similarity_score"], reverse=True)
  return scored_chunks[:top_k]

In [ ]:
def print_answer(question, top_k=5):
  results = find_relevant_chunks(question, chunks, top_k=top_k)
  for idx, r in enumerate(results):
    print(f'result number {idx + 1}:')
    print(f'similarity score: {r["similarity_score"]}')
    print('page:', r['page'])
    print('heading:', r['heading'])
    print(r['text'])
    print('***************************')

In [22]:
print_answer('گشتاور نیرو را چطور تعریف می کنیم؟')

result number 1:
similarity score: 0.6775262496004902
page: 107
heading: 
۹۹ F ۲ M ۲ M ۱ F ۱ d ۲ d ۱) Nm (متر ) است، یکای گشتاور نیرو، نیوتونm () و یکای فاصله مترN (با توجه به اینکه یکای نیرو نیوتون است. های ساده توانیم درک بهتری از اساس کار برخی از ماشین‌اکنون که با گشتاور نیرو آشنا شدیم، می‌دست آوریم.  به بزرگی گشتاور نیرو برابر با حاصل‌ضرب اندازه نیرو در فاصله محل اثر نیرو تا محور چرخش است. بزرگی گشتاور نیرو به اندازۀ نیرو و فاصلۀ نقطه اثر نیرو تا محورچرخش بستگی دارد.۷ شکل محور چرخش فاصله نقطه اثر نیرو تا محور چرخش اندازه نیرو × فاصله نقطه اثر نیرو تا محور چرخش = اندازه گشتاور نیرو‌تر باز کرد؟ توان آسان‌توضیح دهید چرا با آچار بلندتر، مهره محکم را می‌خود را بیازمایید
***************************
result number 2:
similarity score: 0.6652214079996585
page: 91
heading: ها‌کاربرد فسیل
۸ فصل دانید چرا هرچه از تاج سد به پایۀ آن نزدیک کند. آیا می آب جمع شده در پشت سد، فشار زیادی به آن وارد می‌، درک بهتری برای پاسخ ۵۷ یابد؟ با انجام دادن آزمایش کنید صفحه شویم، ضخامت دیوارۀ آن افزایش می می به 

In [23]:
print_answer('انواع باکتری را نام ببرید و توضیح دهید')

result number 1:
similarity score: 0.7085738954360786
page: 134
heading: گوناگونی جانداران
۱۲۶ دقیقه آن را ۲۰ روی قوطی کنسرو مواد غذایی نوشته‌شده است: «قبل از مصرف به مدت را پیدا کنید.  بجوشانید». با مراجعه به منابع معتبر، علت آن آوری اطالعات جمع‌های گوناگونی دارند و در  ها شکل شناسید؟ باکتری هایی از این جانداران می ‌ها: چه ویژگی
***************************
result number 2:
similarity score: 0.6850909505531296
page: 137
heading: جان؟  جاندار یا بی
ها، موجوداتی به  دانید عامل این بیماری ایم. می یک از ما دچار آنفلوآنزا و سرماخوردگی شده  هر شود؛ درحالی‌که خارج از بدن ما  نام ویروس هستند. ویروس سرماخوردگی درون بدن ما تکثیر می‌فعالیت ها  انواعی از قارچ ۱۰ شکل، ها و آغازیان در صنعت غذایی ها، قارچ هایی ازاستفاده باکتری با مراجعه به منابع معتبر مثال تواند  آوری و گزارش آن را در کالس ارائه دهید. هر گروه می کشاورزی، محیط‌زیست و… جمع بخشی از این کار را انجام دهد. ای الف) قارچ‌های پریاخته ب) قارچ مخمر در مشاهده با میکروسکوپ نوری گویند «قارچ از برند. مثال می عنوان نوعی سبزی نام می ها به شود که ازقا

In [24]:
print_answer('تفاوت تندی متوسط و سرعت متوسط چیست؟')

result number 1:
similarity score: 0.5957557179937955
page: 50
heading: تندی متوسط
سال پیش به کمک آزمایش به بررسی و مطالعه چگونگی ۵۰۰ گالیله دانشمند سرشناس ایتالیایی نزدیک به حرکت اجسام پرداخت. مردم تا پیش از گالیله، حرکت اجسام را به صورت «کند» و «تند» توصیف شود. صورت زیر تعریف می‌یک متحرک بود که به ۱ کردند. یکی از کارهای گالیله، معرفی تندی متوسط می‌= تندی متوسط مسافت پیموده شده زمان صرف شده   مدت) ۱ (تندی متوسط خودتان را هنگام رفتن از خانه به مدرسه حساب کنید. اگر با پای پیاده این فاصله متر ۰ / ۴ های خود را از خانه تا مدرسه بشمارید. طول هر قدم را حدود کنید تعداد قدم را طی می‌پیمایید مسافت طی شده را از روی کیلومتر شمار خودرو  بگیرید. اگر با خودرو این فاصله را می‌سنج اندازه بگیرید.  بخوانید. در هر دو حالت زمان طی مسافت را به کمک ساعت یا زمان ثانیه و در اختیار اوسین بولت ۹ / ۵۸، متر مردان ۱۰۰ رکورد جهانی دوی ۱ به نام خود ثبت کرده است. تندی ۲۰۰۹ دونده جامائیکایی است که در سال دست آمده  متوسط این قهرمان جهانی را حساب کنید. مفهوم فیزیکی عدد به را توضیح دهید. فعالیت = تندی متوسط = ۱۴ m / s ۸۴

# 4. Response Generation with LLM

In [25]:
model_name = "SajjadAyoubi/bert-base-fa-qa"
# model_name = "timpal0l/mdeberta-v3-base-squad2"
qa_model = pipeline(
  "question-answering",
  model=model_name,
  tokenizer=model_name
)

config.json:   0%|          | 0.00/626 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/649M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/649M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/324 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.20M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cpu


In [26]:
def generate_answer(question, context):
  result = qa_model(question=question, context=context)
  return {
    "answer": result['answer'],
    "confidence": result['score'],
    "source": context
  }

In [27]:
# q = 'انواع باکتری را نام ببرید و توضیح دهید'
# q = 'تفاوت تندی متوسط و سرعت متوسط چیست؟'
q = 'گشتاور نیرو را چطور تعریف می کنیم؟'

In [28]:
def generate_answer_with_context(question, top_contexts=5):
  relevant_chunks = find_relevant_chunks(question, chunks, top_k=top_contexts)
  context = '\n'.join([item['text'] for item in relevant_chunks])
  answer = generate_answer(question, context)
  return answer, context, relevant_chunks

In [29]:
generated, _0, _1 = generate_answer_with_context(q)
print(generated['confidence'])
print(generated['answer'])

0.2283274084329605
برابر با حاصل‌ضرب اندازه نیرو در فاصله محل اثر نیرو تا محور چرخش


In [30]:
q = 'اجزای تشکیل دهنده هوا چیست؟'
generated, _0, _1 = generate_answer_with_context(q)
print(generated['confidence'])
print(generated['answer'])

0.08747392147779465
به‌اکسیژن یکی از گازهای تشکیل)


In [31]:
q = 'گوگرد چیست؟'
generated, _0, _1 = generate_answer_with_context(q)
print(generated['confidence'])
print(generated['answer'])

0.3635247051715851
گوگرد جامدی S نشانه شیمیایی‌های خاموش  زردرنگ است


In [32]:
q = 'کاربرد فلوئور چیست؟'
generated, _0, _1 = generate_answer_with_context(q)
print(generated['confidence'])
print(generated['answer'])

0.0003698906220961362
برای پختن غذا و سیم‌گسترده‌های دیگری از مس و  باشند.


In [33]:
def get_reliable_answer(question, min_confidence=0.3):
  answer, context, relevant_chunks = generate_answer_with_context(question)
  if answer and answer['confidence'] >= min_confidence:
    return answer
  else:
    # best matching chunk
    best_chunk = relevant_chunks[0]
    return {
      "answer": best_chunk['text'],
      "confidence": 0.99,
      "source": best_chunk.get('heading'),
      "fallback": True
    }

In [49]:
q = 'ارتباطات همزیستی بین جانداران چند نوع است؟'
get_reliable_answer(q)

{'answer': 'سه نوع ارتباط همزیستی، شکار و شکارچی و رقابت',
 'confidence': 0.4750603139400482,
 'source': 'توان \u200aسازگان سه نوع ارتباط همزیستی، شکار و شکارچی و رقابت را می\u200aبین جانداران در هر بوم تشخیص داد. قارچ روی تنه درخت رشد کرده است.۳ شکل\n۱۲۶ دقیقه آن را ۲۰ روی قوطی کنسرو مواد غذایی نوشته\u200cشده است: «قبل از مصرف به مدت را پیدا کنید. \u200aبجوشانید». با مراجعه به منابع معتبر، علت آن آوری اطالعات\u200aجمع\u200cهای گوناگونی دارند و در \u200aها شکل\u200aشناسید؟ باکتری\u200aهایی از این جانداران می\u200a\u200cها: چه ویژگی\nشناسی مختلفی مانند\u200cهای زمین\u200aدر هنگام عبور از نواحی کوهستانی البرز و زاگرس با پدیده\u200cهایی در \u200aشوید. شاید چنین پرسش\u200cها و… مواجه می\u200cهای کوچک و بزرگ، شکستگی\u200cخوردگی\u200cچین اند؟ قبل از تشکیل آنها سرزمین ایران چه وجود آمده\u200cها چگونه به\u200cکوه\u200cذهنتان ایجاد شود که این رشته\u200cها چیست؟ با مطالعه این فصل، پاسخ \u200aها و شکستگی\u200aخوردگی\u200aشکلی بوده است؟ علت پیدایش چین آورید. \u200aها را به دست می\u200cاین پرسش ۶ ف